# PySHACL: SHACL in Python

see https://github.com/RDFLib/pySHACL


## Preparations

install and import rdflib and pyshacl

In [17]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

In [18]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)


## Task 1
Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.

In [19]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .     
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

<PersonKnowsShape>
    sh:path :knows ;
    sh:nodeKind sh:IRI.

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,n211a3236862f46158f95ba52d7cb72a1b1,sh:NodeKindConstraintComponent,:PersonKnowsShape,Value is not of Node Kind sh:IRI


## Task 2
Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.

In [20]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<ManShape> a sh:NodeShape ;
    sh:targetClass <Man> ;
    sh:and ([sh:class <Man> ; ][sh:not [sh:class <Woman> ; ]]
             ).


""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John does not conform to all shapes in [ sh:class :Man ] , [ sh:not [ sh:class :Woman ] ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:AndConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :John ],
        [ a sh:ValidationResult ;
            sh:focusNode :Susi ;
            sh:resultMessage "Node :Susi does not conform to all shapes in [ sh:class :Man ] , [ sh:not [ sh:class :Woman ] ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:AndConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :Susi ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,None,:John,sh:AndConstraintComponent,:ManShape,Node :John does not conform to all shapes in [...
1,:Susi,None,:Susi,sh:AndConstraintComponent,:ManShape,Node :Susi does not conform to all shapes in [...


## Task 3
Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [21]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

<PersonKnowsShape>
    sh:path :knows ;
    sh:class <Person> .



""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Mary" ] ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,nd91b851e071f4cd1b88aa02be512eb77b1,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person
1,:Peter,:knows,:Mary,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person


## Task 4
Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive

In [22]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonAgeShape> .

<PersonAgeShape>
    sh:path :age;
    sh:minInclusive 0 ;
    sh:maxInclusive 150 ;
    sh:maxCount 1 ;
    sh:minCount 1 .


<AdultShape> a sh:NodeShape ;
    sh:targetClass <Adult> ;
    sh:property <AdultAgeShape> .

<AdultAgeShape>
    sh:path :age ;
    sh:minInclusive 19 .

<SeniorShape> a sh:NodeShape ;
    sh:targetClass <Senior> ;
    sh:property <SeniorAgeShape> .

<SeniorAgeShape>
    sh:path :age ;
    sh:minInclusive 65 .

<ChildShape> a sh:NodeShape ;
    sh:targetClass <Child> ;
    sh:property <ChildAgeShape> .

<ChildAgeShape>
    sh:path :age ;
    sh:maxInclusive 12 .




""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bibo ;
            sh:resultMessage "More than 1 values on :Bibo->:age" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxCountConstraintComponent ;
            sh:sourceShape :PersonAgeShape ],
        [ a sh:ValidationResult ;
            sh:focusNode :Beny ;
            sh:resultMessage "Value is not <= Literal(\"12\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxInclusiveConstraintComponent ;
            sh:sourceShape :ChildAgeShape ;
            sh:value 13 ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Val

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Mary,:age,17,sh:MinInclusiveConstraintComponent,:AdultAgeShape,"Value is not >= Literal(""19"", datatype=xsd:int..."
1,:Beny,:age,13,sh:MaxInclusiveConstraintComponent,:ChildAgeShape,"Value is not <= Literal(""12"", datatype=xsd:int..."
2,:Lili,:age,-1,sh:MinInclusiveConstraintComponent,:PersonAgeShape,"Value is not >= Literal(""0"", datatype=xsd:inte..."
3,:Bibo,:age,None,sh:MaxCountConstraintComponent,:PersonAgeShape,More than 1 values on :Bibo->:age
4,:Peter,:age,None,sh:MinCountConstraintComponent,:PersonAgeShape,Less than 1 values on :Peter->:age
5,:John,:age,151,sh:MaxInclusiveConstraintComponent,:PersonAgeShape,"Value is not <= Literal(""150"", datatype=xsd:in..."
6,:Senior2,:age,64,sh:MinInclusiveConstraintComponent,:SeniorAgeShape,"Value is not >= Literal(""65"", datatype=xsd:int..."


## Task 5
Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [23]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <NameShape> ;
    sh:closed true;
    sh:ignoredProperties ( rdf:type ) .

<NameShape>
    sh:path :name;
    sh:datatype xsd:string;
    sh:maxCount 1 ;
    sh:nodeKind sh:Literal .




""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape :NameShape ;
            sh:value :Jim ],
        [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John is closed. It cannot have value: :Mary" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClosedConstraintComponent ;
            sh:sourceShape :PersonShape ;
            sh:value :Mary ],
        [ a sh:ValidationResult ;
            sh:focusNode :Bibi ;
     

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Jane,:name,2343,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string
1,:Bibi,:name,None,sh:MaxCountConstraintComponent,:NameShape,More than 1 values on :Bibi->:name
2,:Jim,:name,:Jim,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string
3,:Jim,:name,:Jim,sh:NodeKindConstraintComponent,:NameShape,Value is not of Node Kind sh:Literal
4,:John,:knows,:Mary,sh:ClosedConstraintComponent,:PersonShape,Node :John is closed. It cannot have value: :Mary


## Task 6
Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet. 
##Kommentar:
Kommentar: Aufgrund der “inCounstraintComponent” Violation bei Salzburg musste ich das „InAustriaShape“ auf diese Art umsetzen.

In [24]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<CityShape> a sh:NodeShape ;
    sh:targetClass <City> ;
    sh:property <InCountryShape> .

<InCountryShape>
    sh:path :inCountry;
    sh:class <Country> .

<EuropeanCityShape> a sh:NodeShape ;
    sh:targetClass <EuropeanCity>;
    sh:property <InEuropeanCountryShape> .

<InEuropeanCountryShape>
    sh:path :inCountry ;
    sh:class <EuropeanCountry> .

<AustrianCityShape> a sh:NodeShape ;
    sh:targetClass <AustrianCity> ;
    sh:property <InAustriaShape> .

<InAustriaShape>
    sh:path :inCountry ;
    sh:in (<Austria>) ;
    sh:closed true ;
    sh:ignoredProperties (rdf:type :inCountry) .





""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Salzburg ;
            sh:resultMessage "Value :Germany not in list [':Austria']" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:InConstraintComponent ;
            sh:sourceShape :InAustriaShape ;
            sh:value :Germany ],
        [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :EuropeanCountry" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :InEuropeanCountryShape ;
            sh:value :France ],
        [ a sh:ValidationResult ;
            sh:focusN

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Salzburg,:inCountry,:Germany,sh:InConstraintComponent,:InAustriaShape,Value :Germany not in list [':Austria']
1,:NewYork,:inCountry,:USA,sh:ClassConstraintComponent,:InCountryShape,Value does not have class :Country
2,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:InCountryShape,Value does not have class :Country
3,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:InEuropeanCountryShape,Value does not have class :EuropeanCountry


##Task 7
Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes. 

In [25]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<WorksForSubjectShape>
    sh:targetSubjectsOf :worksFor ;
    sh:class <Person> .

<WorksForObjectShape>
    sh:targetObjectsOf :worksFor ;
    sh:class <Organization> .





""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :LMU ;
            sh:resultMessage "Value does not have class :Organization" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForObjectShape ;
            sh:value :LMU ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForSubjectShape ;
            sh:value :Jim ],
        [ a sh:ValidationResult ;
            sh:focusNode :Mary ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:r

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:LMU,None,:LMU,sh:ClassConstraintComponent,:WorksForObjectShape,Value does not have class :Organization
1,:Mary,None,:Mary,sh:ClassConstraintComponent,:WorksForSubjectShape,Value does not have class :Person
2,:Jim,None,:Jim,sh:ClassConstraintComponent,:WorksForSubjectShape,Value does not have class :Person


##Task 8
Peter kennt ausschließlich Männer, die in Österreich leben. 

##Task 9
Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals

In [26]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ProductShape> a sh:NodeShape ;
    sh:targetClass <Product> ;
    sh:property [
       sh:path :netWeight ;
       sh:lessThanOrEquals :grossWeight
].





""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bread ;
            sh:resultMessage "Value of :Bread->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bread,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,n5a8eb0d9a72d46c7982e5a792694ef55b1,"Value of :Bread->:grossWeight < Literal(""1.2"",..."


## Task 10
Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine 